# Mesh Refinement Tutorial - Large Deformation Curved Beam

This tutorial covers the meshing and simulation of the deflection of a curved beam, modeled by offset archimedean spirals.

Here we define a slight modification of the labels_plot_mesh_2D function in pre_preocess_demo_helper_fcns in order to plot fine meshes without labels.

In [1]:
import os
import warnings
warnings.simplefilter("always")
from finiteelementanalysis import pre_process as pre
from finiteelementanalysis import pre_process_demo_helper_fcns as pre_demo
from finiteelementanalysis.solver import hyperelastic_solver
from finiteelementanalysis import visualize as viz
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

def custom_no_labels_plot_mesh_2D(fname: str, ele_type: str, coords: np.ndarray, connect: np.ndarray, gauss_points: np.ndarray = None):
    """
    Plots a 2D finite element mesh with aesthetically pleasing visualization.
    
    Parameters
    ----------
    fname : str
        The filename for saving the plot.
    ele_type : str
        The type of finite element.
    coords : np.ndarray of shape (n_nodes, 2)
        The coordinates of the nodes in physical space.
    connect : np.ndarray of shape (n_elements, n_nodes_per_element)
        The element connectivity matrix, specifying node indices for each element.
    gauss_points : np.ndarray of shape (num_elements, num_gauss_pts, 2), optional
        The physical coordinates of Gauss points for visualization.
    """
    fig, ax = plt.subplots(figsize=(9, 9), dpi=150)

    for elem_idx, element in enumerate(connect):
        element_coords = coords[element]

        if ele_type == "D2_nn3_tri":
            edges = [[0, 1], [1, 2], [2, 0]]
        elif ele_type == "D2_nn6_tri":
            edges = [[0, 3], [3, 1], [1, 4], [4, 2], [2, 5], [5, 0]]
        elif ele_type == "D2_nn4_quad":
            edges = [[0, 1], [1, 2], [2, 3], [3, 0]]
        elif ele_type == "D2_nn8_quad":
            edges = [[0, 4], [4, 1], [1, 5], [5, 2], [2, 6], [6, 3], [3, 7], [7, 0]]
        else:
            raise ValueError(f"Unsupported element type: {ele_type}")

        # Draw element edges in a soft gray
        for edge in edges:
            ax.plot(element_coords[edge, 0], element_coords[edge, 1], color='gray', lw=0.8, alpha=0.7)

    # Plot nodes
    ax.scatter(coords[:, 0], coords[:, 1], color=(0.8, 0.8, 0.8), s=20, edgecolors='red', linewidth=0.5, zorder=3)

    # Plot Gauss points if provided
    if gauss_points is not None:
        gauss_x = gauss_points[:, :, 0].flatten()  # Extract all x-coordinates
        gauss_y = gauss_points[:, :, 1].flatten()  # Extract all y-coordinates
        ax.scatter(gauss_x, gauss_y, color='#AFCBFF', marker='o', s=15, edgecolors='black', linewidth=0.3, zorder=2, alpha=0.8, label="Gauss Points")

    ax.set_xlabel("X Coordinate", fontsize=12)
    ax.set_ylabel("Y Coordinate", fontsize=12)
    ax.set_title(f"2D Mesh Plot for {ele_type}", fontsize=14, fontweight='bold')
    ax.set_aspect('equal')
    ax.grid(False)  # Remove grid for a cleaner look

    # Add legend if Gauss points are plotted
    if gauss_points is not None:
        ax.legend(loc='upper right', fontsize=10, frameon=False)

    # Save with high DPI and clean style
    plt.savefig(fname, dpi=300, bbox_inches='tight', pad_inches=0.1)
    plt.close()
    return

## Geometry Setup and Mesh Generation

To demonstrate the effects of mesh refinement, we generate 3 meshes - a course mesh with 2D_nn3_tri elements, a fine mesh with the same 2D_nn3_tri elements, and a course mesh with 2D_nn6_tri elements.

In [2]:
# for saving files later
tutorials_dir = os.path.abspath('')

# FEA problem info
ndof = 2
ele_types = ["D2_nn3_tri", "D2_nn6_tri"]
mesh_sizes = [0.1, 0.05, 0.025]

# Geometric parameters defining spiral outline
N_layers = 3
r_0 = 0.2
dr_archimedes = 0.2
dr = 0.1

theta_end = 2*np.pi*N_layers
L_archimedes = dr_archimedes/(4*np.pi) * (theta_end*np.sqrt(1 + theta_end**2) + np.log(theta_end + np.sqrt(1 + theta_end**2)))

# Create meshes for different combinations of element type and mesh size
coord_list = []
connect_list = []
for ele_type in ele_types:
    for i_mesh, mesh_size in enumerate(mesh_sizes):
        mesh_name = f"archimedean_spiral_mesh_{ele_type}_{i_mesh}"

        # Calculate outline
        theta = np.linspace(0, theta_end, int(np.ceil(L_archimedes/mesh_size)))
        rad = np.linspace(0, dr, int(np.ceil(dr/mesh_size)))
        # Inner spiral
        x_archimedes_1 = (dr_archimedes/(2*np.pi) * theta + r_0) * np.cos(theta)
        y_archimedes_1 = (dr_archimedes/(2*np.pi) * theta + r_0) * np.sin(theta)
        # Straight line to outer spiral
        x_archimedes_2 = dr_archimedes * N_layers + r_0 + rad
        y_archimedes_2 = np.zeros(len(rad))
        # Outer spiral (reversed)
        x_archimedes_3 = (dr_archimedes/(2*np.pi) * theta + r_0 + dr) * np.cos(theta)
        y_archimedes_3 = (dr_archimedes/(2*np.pi) * theta + r_0 + dr) * np.sin(theta)
        # Straight line to inner spiral (reversed)
        x_archimedes_4 = r_0 + rad
        y_archimedes_4 = np.zeros(len(rad))
        # Combine boundaries
        x_archimedes = np.hstack((x_archimedes_1, x_archimedes_2, np.flip(x_archimedes_3), np.flip(x_archimedes_4)))
        y_archimedes = np.hstack((y_archimedes_1, y_archimedes_2, np.flip(y_archimedes_3), np.flip(y_archimedes_4)))
        outline_points = [[x_archimedes[i_pnt], y_archimedes[i_pnt]] for i_pnt in range(len(x_archimedes))]

        if i_mesh == len(mesh_sizes) and ele_type == "D2_nn3_tri":
            plt.figure(figsize=(8, 6))
            plt.plot(x_archimedes, y_archimedes)
            img_fname = tutorials_dir + "/b-spiral_beam_outline.png"
            plt.savefig(str(img_fname))

        coords, connect = pre.mesh_outline(outline_points, ele_type, mesh_name, mesh_size)
        coord_list.append(coords)
        connect_list.append(connect)

        mesh_img_fname = tutorials_dir + f"/b-spiral_beam_mesh_{ele_type}_{i_mesh}.png"
        #pre_demo.plot_mesh_2D(str(mesh_img_fname), ele_type, coords, connect)
        custom_no_labels_plot_mesh_2D(mesh_img_fname, ele_type, coords, connect)


Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 10%] Meshing curve 2 (Line)
Info    : [ 10%] Meshing curve 3 (Line)
Info    : [ 10%] Meshing curve 4 (Line)
Info    : [ 10%] Meshing curve 5 (Line)
Info    : [ 10%] Meshing curve 6 (Line)
Info    : [ 10%] Meshing curve 7 (Line)
Info    : [ 10%] Meshing curve 8 (Line)
Info    : [ 10%] Meshing curve 9 (Line)
Info    : [ 10%] Meshing curve 10 (Line)
Info    : [ 10%] Meshing curve 11 (Line)
Info    : [ 10%] Meshing curve 12 (Line)
Info    : [ 20%] Meshing curve 13 (Line)
Info    : [ 20%] Meshing curve 14 (Line)
Info    : [ 20%] Meshing curve 15 (Line)
Info    : [ 20%] Meshing curve 16 (Line)
Info    : [ 20%] Meshing curve 17 (Line)
Info    : [ 20%] Meshing curve 18 (Line)
Info    : [ 20%] Meshing curve 19 (Line)
Info    : [ 20%] Meshing curve 20 (Line)
Info    : [ 20%] Meshing curve 21 (Line)
Info    : [ 20%] Meshing curve 22 (Line)
Info    : [ 20%] Meshing curve 23 (Line)
Info    : [ 20%] Meshing curve 24 (Line)
I

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 10%] Meshing curve 2 (Line)
Info    : [ 10%] Meshing curve 3 (Line)
Info    : [ 10%] Meshing curve 4 (Line)
Info    : [ 10%] Meshing curve 5 (Line)
Info    : [ 10%] Meshing curve 6 (Line)
Info    : [ 10%] Meshing curve 7 (Line)
Info    : [ 10%] Meshing curve 8 (Line)
Info    : [ 10%] Meshing curve 9 (Line)
Info    : [ 10%] Meshing curve 10 (Line)
Info    : [ 10%] Meshing curve 11 (Line)
Info    : [ 10%] Meshing curve 12 (Line)
Info    : [ 10%] Meshing curve 13 (Line)
Info    : [ 10%] Meshing curve 14 (Line)
Info    : [ 10%] Meshing curve 15 (Line)
Info    : [ 10%] Meshing curve 16 (Line)
Info    : [ 10%] Meshing curve 17 (Line)
Info    : [ 10%] Meshing curve 18 (Line)
Info    : [ 10%] Meshing curve 19 (Line)
Info    : [ 10%] Meshing curve 20 (Line)
Info    : [ 10%] Meshing curve 21 (Line)
Info    : [ 10%] Meshing curve 22 (Line)
Info    : [ 10%] Meshing curve 23 (Line)
Info    : [ 10%] Meshing curve 24 (Line)
I

The outline of our beam forms a spiral.

<img src="b-spiral_beam_outline.png" alt="Boundaries of Spiral Beam" width="400"/>

Here are all of the generated meshes, using different element types and resolution combinations.

<img src="b-spiral_beam_mesh_D2_nn3_tri_0.png" alt="Mesh 1 of Spiral Beam" width="400"/>
<img src="b-spiral_beam_mesh_D2_nn3_tri_1.png" alt="Mesh 2 of Spiral Beam" width="400"/>
<img src="b-spiral_beam_mesh_D2_nn3_tri_2.png" alt="Mesh 3 of Spiral Beam" width="400"/>
<img src="b-spiral_beam_mesh_D2_nn3_tri_3.png" alt="Mesh 4 of Spiral Beam" width="400"/>
<img src="b-spiral_beam_mesh_D2_nn3_tri_4.png" alt="Mesh 5 of Spiral Beam" width="400"/>

<img src="b-spiral_beam_mesh_D2_nn6_tri_0.png" alt="Mesh 6 of Spiral Beam" width="400"/>
<img src="b-spiral_beam_mesh_D2_nn6_tri_1.png" alt="Mesh 7 of Spiral Beam" width="400"/>
<img src="b-spiral_beam_mesh_D2_nn6_tri_2.png" alt="Mesh 8 of Spiral Beam" width="400"/>
<img src="b-spiral_beam_mesh_D2_nn6_tri_3.png" alt="Mesh 9 of Spiral Beam" width="400"/>
<img src="b-spiral_beam_mesh_D2_nn6_tri_4.png" alt="Mesh 10 of Spiral Beam" width="400"/>



## Evaluate Quality of Meshes

In [ ]:
# Create element quality historams
for i_type, ele_type in enumerate(ele_types):
    for i_size, mesh_size in enumerate(mesh_sizes):
        mesh_ind = len(mesh_sizes) * i_type + i_size
        mesh_name = f"archimedean_spiral_mesh_{ele_type}_{i_mesh}"
        coords = coord_list[mesh_ind]
        connect = connect_list[mesh_ind]
        aspect_ratios, skewness, min_angles, max_angles = pre_demo.compute_element_quality_metrics(ele_type, coords, connect)
        cond_nums, jac_dets = pre_demo.compute_condition_and_jacobian(ele_type, coords, connect)
        plot_file = mesh_name + f"_histograms"
        pre_demo.plot_element_quality_histograms(
                fname=str(plot_file),
                super_title="Test Mesh Quality Metrics (%s)" % (ele_type),
                ele_type=ele_type,
                cond_nums=cond_nums,
                jac_dets=jac_dets,
                aspect_ratios=aspect_ratios,
                skewness=skewness,
                min_angles=min_angles,
                max_angles=max_angles
            )

## Apply Boundary Conditions and Perscribed Displacements

Here we hold the horizontal edge in the center of the spiral fixed, while applying a small, constant, downward velocity on the horizontal outside end of the spiral.

In [ ]:
# Identify boundaries for each mesh and build a constraint list
v_y = -0.1
tol = 10**-6
moving_node_list = []
constraint_list = []
for i_mesh in range(2*len(mesh_sizes)):
    coords = coord_list[i_mesh]
    fixed_boundary = set()
    moving_boundary = set()
    for i_node in range(coords.shape[0]):
        xval, yval = coords[i_node]
        if abs(yval) < tol:
            if xval >= r_0 - tol and xval <= r_0 + dr + tol:
                fixed_boundary.add(i_node)
                constraints.append((i_node, 0, 0))
                constraints.append((i_node, 1, 0))
            elif xval >= dr_archimedes * N_layers + r_0 - tol and xval <= dr_archimedes * N_layers + r_0 + rad + tol:
                moving_boundary.add(i_node)
                constraints.append((i_node, 1, v_y))
    moving_node_list.append(moving_boundary)
    # Add to constraint list in proper format
    constraint_list.append(np.array(constraints_1, dtype=float).T)

# No distributed load is applied
dload_info = np.empty((ndof + 2, 0))

# Choose material properties
material_props = np.array([134.6, 83.33])  # [mu, K]

## Run the Solver for each Mesh

In [ ]:
# Number of incremental loading steps
nr_num_steps = 50
disp_list = []
for i_mesh in range(2*len(mesh_sizes)):
    ele_type = ele_types[int(i_mesh / len(mesh_sizes)]
    coords = coord_list[i_mesh]
    connect = connect_list[i_mesh]
    fixed_nodes = constraint_list[i_mesh]
    # Run the solver
    displacements_all, nr_info_all = hyperelastic_solver(
        material_props,
        ele_type,
        coords.T,      # solver expects coords as (ncoord, n_nodes)
        connect.T,     # and connectivity as (n_nodes_per_elem, n_elems)
        fixed_nodes,
        dload_info,
        nr_print=True,
        nr_num_steps=nr_num_steps,
        nr_tol=1e-8,
        nr_maxit=30,
    )
    disp_list.append(displacements_all)

## Create Animations of Results

In [ ]:
for i_mesh in range(2*len(mesh_sizes)):
    ele_type = ele_types[int(i_mesh / len(mesh_sizes))]
    coords = coord_list[i_mesh]
    connect = connect_list[i_mesh]
    # Save an animation of the deformation
    img_name = f"/b-deformation_{ele_type}_{i_mesh % len(mesh_sizes)}.gif"
    fname = tutorials_dir + img_name
    viz.make_deformation_gif(disp_list[i_mesh] + list(reversed(disp_list[i_mesh])), coords, connect, ele_type, fname, interval=40, magnification=2)


<img src="b-deformation_D2_nn3_tri_0.gif" alt="Deformation of Mesh 1 of Spiral Beam" width="400"/>
<img src="b-deformation_D2_nn3_tri_1.gif" alt="Deformation of Mesh 2 of Spiral Beam" width="400"/>
<img src="b-deformation_D2_nn3_tri_2.gif" alt="Deformation of Mesh 3 of Spiral Beam" width="400"/>
<img src="b-deformation_D2_nn3_tri_3.gif" alt="Deformation of Mesh 4 of Spiral Beam" width="400"/>
<img src="b-deformation_D2_nn3_tri_4.gif" alt="Deformation of Mesh 5 of Spiral Beam" width="400"/>

<img src="b-deformation_D2_nn3_tri_5.gif" alt="Deformation of Mesh 6 of Spiral Beam" width="400"/>
<img src="b-deformation_D2_nn3_tri_6.gif" alt="Deformation of Mesh 7 of Spiral Beam" width="400"/>
<img src="b-deformation_D2_nn3_tri_7.gif" alt="Deformation of Mesh 8 of Spiral Beam" width="400"/>
<img src="b-deformation_D2_nn3_tri_8.gif" alt="Deformation of Mesh 9 of Spiral Beam" width="400"/>
<img src="b-deformation_D2_nn3_tri_9.gif" alt="Deformation of Mesh 10 of Spiral Beam" width="400"/>

In [ ]:
x_vals_2D_nn3_tri = []
x_vals_2D_nn6_tri = []
for i_mesh in range(2*len(mesh_sizes)):
    moving_nodes = sorted(moving_node_list[i_mesh], key=lambda i: coords[i, 0])  # sort by x-coordinate
    final_x_disp = disp_list[i_mesh][-1][moving_nodes[0]]
    if i_mesh < len(mesh_sizes):
        x_vals_2D_nn3_tri.append(final_x_disp)
    else:
        x_val_2D_nn6_tri.append(final_x_disp)

# Plot the x values computed for each mesh size and element type
plt.figure(figsize=(8, 6))
plt.plot(mesh_sizes, x_vals_2D_nn3_tri, 'ro-', label="2D_nn3_tri computed x at end")
plt.plot(mesh_sizes, x_vals_2D_nn6_tri, 'b--', label="2D_nn6_tri computed x at end")
plt.xlabel("mesh size")
plt.ylabel("computed x of end node")
plt.title("Comparison of computed x values")
plt.legend()
plt.grid(True)
plt.tight_layout()

# Save the plot image
img_fname = tutorials_dir + "/b-mesh-refinement-comparison.png"
plt.savefig(str(img_fname))



<img src="b-mesh-refinement-comparison.png" alt="Comparison of computed x-displacements at end point" width="400"/>

This graph should show the convergence of the computed value with decreasing mesh size (finer meshes), with the 2D_nn6_tri curve much closer to the converged value than the 2D_nn3_tri curve for any of the chosen mesh sizes. This shows how p-refinement (using higher-order elements), can sometimes provide better convergence for the same increase in computational time, compared to using h-refinement (using a finer mesh).

![](b-deformation_base.gif "Deformation Result")

![](b-deformation_h-refinement.gif "Deformation Result - h-refinement")

![](b-deformation_p-refinement.gif "Deformation Result - p-refinement")